# Análisis de sentimientos en el ámbito sociopolítico del Ecuador

## Importar módulos

In [1]:
from core.login_twitter import login_twitter
from core.dirver_manager import get_driver, close_driver
from core.utils import limpiar_nombre_archivo, crear_carpeta_si_no_existe
from scraping.enlaces_scraper import guardar_enlaces_candidatos
from scraping.tweets_scraper import extraer_informacion_tweets
from scraping.comentarios_scraper import extraer_enlaces_comentarios_csvs, scroll_comentarios
from scraping.contenido_comentarios_scraper import minar_texto_comentarios
from preprocessing.consolidar_corpus import consolidar_corpus_csvs
from preprocessing.limpieza_texto import limpiar_y_normalizar_corpus
from preprocessing.reestructurar_csv_a_json import reestructurar_csv_a_json_simple
from dotenv import load_dotenv
from pathlib import Path
import os
import time

## Definición de las cadenas de consulta

In [ ]:
candidatos_presidenciales_2025 = [
    #"Daniel Noboa", "Luisa González", "Wilson Gómez Vascones", "Jimmy Jairala",
    "Jorge Escala", "Andrea González Náder", "Henry Kronfle", "Carlos Rabascall", 
    "Leonidas Iza", "Iván Saquicela", "Francesco Tabacchi", "Henry Cucalón",
    "Enrique Gómez", "Víctor Araus", "Juan Iván Cueva", "Pedro Granja"
]


palabras_clave = [
    "elecciones 2025"
    # , "presidenciales 2025", "candidato presidencial", "candidatura 2025",
    # "campaña presidencial", "Ecuador 2025", "presidente Ecuador", "voto 2025",
    # "debate presidencial", "elecciones Ecuador"
]

## Configuración del driver

Al usar get_driver logramos que el web driver no se cierre lo que nos permitirá usar el mismo para realizar las acciones de web scraping

In [2]:
driver = get_driver()

## Llamadas a las variables de entorno

In [3]:
# 1. Obtenemos el Directorio de Trabajo Actual (donde se ejecuta el Notebook)
#    Si ejecutas desde 'proyecto/src', esto devolverá 'proyecto/src'
current_dir = Path(os.getcwd())

# 2. Subimos un nivel para llegar a la carpeta 'proyecto/' (donde está .env)
#    Esto cambia la ruta de 'proyecto/src' a 'proyecto'
base_dir = current_dir.parent

# 3. Construimos la ruta completa al archivo .env
dotenv_path = base_dir / '.env'

# --- Carga de Variables ---
load_dotenv(dotenv_path=dotenv_path)

# --- Llamada de Variables ---
username = os.getenv("X_USERNAME")
password = os.getenv("X_PASSWORD")
email = os.getenv("X_EMAIL")

## Inicio de sesión

Nos ayudará a mantener iniciada la sesión y evitar varios logeos

In [4]:
login_twitter(driver, username, password, email)

Paso 1: Ingresando Correo con simulación de tecleo...
Paso 4: Ingresando Contraseña con simulación de tecleo...
✅ Inicio de sesión completado. Verificando redirección...


## Minar enlaces de las publicaciones en base a las cadenas de consulta

La cadena de consulta se compone de la unión de candidatos_presidenciales_2025 y palabras_claves
Por ejemplo: "Daniel Noboa elecciones 2025"
Una ve minados todos los enlaces a las publicaciones que aparecen en esta consulta, se guardarán los enlces en `Daniel_Noboa_enlaces_tweets.csv`, siendo "enlaces_tweets" el subijo para cada csv correspondiente a cada candidato

In [ ]:
guardar_enlaces_candidatos(driver, candidatos_presidenciales_2025, palabras_clave)

## Minar contenido de cada publicación

In [6]:
extraer_informacion_tweets(driver, "datasets/raw/enlaces_publicaciones_v2", "datasets/raw/informacion_publicaciones")

Error en https://x.com/HenryKronfle/status/1256406173986717698/media_tags: Message: 
Stacktrace:
	GetHandleVerifier [0x0xc0fe43+66515]
	GetHandleVerifier [0x0xc0fe84+66580]
	(No symbol) [0x0x9fdc48]
	(No symbol) [0x0xa48704]
	(No symbol) [0x0xa48aab]
	(No symbol) [0x0xa8f482]
	(No symbol) [0x0xa6b214]
	(No symbol) [0x0xa8cba7]
	(No symbol) [0x0xa6afc6]
	(No symbol) [0x0xa3c2ca]
	(No symbol) [0x0xa3d154]
	GetHandleVerifier [0x0xe67353+2521315]
	GetHandleVerifier [0x0xe622d3+2500707]
	GetHandleVerifier [0x0xc37c94+229924]
	GetHandleVerifier [0x0xc281f8+165768]
	GetHandleVerifier [0x0xc2ecad+193085]
	GetHandleVerifier [0x0xc18158+100072]
	GetHandleVerifier [0x0xc182f0+100480]
	GetHandleVerifier [0x0xc025aa+11066]
	BaseThreadInitThunk [0x0x764e5d49+25]
	RtlInitializeExceptionChain [0x0x7783d6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x7783d661+561]

Error en https://x.com/Informatech_ec/status/1888656996905218421/media_tags: Message: 
Stacktrace:
	GetHandleVerifier [0x0xc0fe43+66515]
	

## Minar enlaces de los comentarios de cada publicación

In [5]:
extraer_enlaces_comentarios_csvs(driver, "datasets/raw/informacion_publicaciones_v2", "datasets/raw/enlaces_comentarios_publicaciones")

🧵 0 comentarios en https://x.com/elmercuriomanta/status/1758883471294902388
🧵 0 comentarios en https://x.com/oromartv/status/1823517796073976001
🧵 0 comentarios en https://x.com/PedritoExtranja/status/1940041041882603747
🧵 0 comentarios en https://x.com/byronbecb/status/1884601734992961826
🧵 22 comentarios en https://x.com/PedritoExtranja/status/1524059052862496771
🧵 2 comentarios en https://x.com/PilarGGranja/status/1951357208652066958
🧵 8 comentarios en https://x.com/PedritoExtranja/status/1805722338542940628/photo/1
🧵 2 comentarios en https://x.com/lahistoriaec/status/1806905308050051175
🧵 24 comentarios en https://x.com/ElDatoEcua/status/1879214999081488627
🧵 2 comentarios en https://x.com/GKecuador/status/1881809746128437585
🧵 4 comentarios en https://x.com/MarcoAnibal/status/1826261948226310619
🧵 2 comentarios en https://x.com/EcuadorTV/status/1794877097174867986/photo/1
🧵 6 comentarios en https://x.com/PedritoExtranja/status/1886869695938355255
🧵 2 comentarios en https://x.com/r

## Minar contenido comentarios de cada enlace

In [ ]:
minar_texto_comentarios(driver, "datasets/raw/enlaces_comentarios_publicaciones", "datasets/raw/datos_completos_publicacion", limite=None)


--- Procesando archivo: Daniel_Noboa_enlaces_comentarios.csv (2719 comentarios/filas) ---
[3/2719] Extraído: https://x.com/juanG_8814/status/19240667...
[4/2719] Extraído: https://x.com/juanG_8814/status/19240667...
[5/2719] No se pudo extraer texto de: https://x.com/Alberto41767748/status/192... Error: TimeoutException
[6/2719] Extraído: https://x.com/juanG_8814/status/19240761...
[7/2719] No se pudo extraer texto de: https://x.com/Alberto41767748/status/192... Error: TimeoutException
[8/2719] No se pudo extraer texto de: https://x.com/Alberto41767748/status/192... Error: TimeoutException
[9/2719] Extraído: https://x.com/juanG_8814/status/19240761...
[10/2719] Extraído: https://x.com/MichaelJairo/status/190400...
[11/2719] Extraído: https://x.com/DuduDudu701/status/1904132...
[12/2719] Extraído: https://x.com/MichaelJairo/status/190400...
[13/2719] Extraído: https://x.com/Cristia35551879/status/190...
[14/2719] Extraído: https://x.com/gsalvador1969/status/19042...
[15/2719] Extraído:

## Cerrar driver

In [6]:
close_driver(driver)

## Unir CSV candidatos

In [3]:
consolidar_corpus_csvs("datasets/raw/datos_completos_publicacion", "datasets/raw/corpus_completo/corpus_comentarios_tweets.csv")

Iniciando consolidación de CSVs en: datasets/raw/datos_completos_publicacion
Cargando: Daniel_Noboa_datos_completos.csv

Cargados 1 archivos. Concatenando...
Total de filas en el corpus: 1435
Corpus guardado en: datasets/raw/corpus_completo/corpus_comentarios_tweets.csv


## Limpiar CSV

In [4]:
limpiar_y_normalizar_corpus("datasets/raw/corpus_completo/corpus_comentarios_tweets.csv", "datasets/processed")

Archivo cargado. Filas iniciales: 1435
Duplicados eliminados. Filas restantes: 655 (Eliminados: 780)
Filas nulas eliminadas. Filas restantes: 233 (Eliminados: 422)

Aplicando limpieza y normalización...

Datos limpiados y guardados en 'datasets/processed\corpus_preprocesado.csv'


## Convertir CSV a JSON

In [5]:
reestructurar_csv_a_json_simple("datasets/processed/corpus_preprocesado.csv", "datasets/processed/corpus.json")

Archivo CSV cargado: 233 filas.
Reestructurando datos...

Archivo reestructurado guardado como 'datasets/processed/corpus.json'
Total de publicaciones (posts) únicas reestructuradas: 26
